In [1]:
import requests
import json


In [2]:
response_API = requests.get('https://developer.nps.gov/api/v1/roadevents?type=workzone&api_key=pAVN008JKe6w3BaWr6FmH8jPFbzkAbKkv6ZpditQ')
#print(response_API.status_code)


In [3]:
data = response_API.text



In [12]:
data

'{"road_event_feed_info":{"publisher":"National Park Service","version":"4.1","update_date":"2023-04-28T06:37:22.497618Z","update_frequency":60,"contact_name":"National Park Service","contact_email":"asknps@nps.gov","data_sources":[{"data_source_id":"40ce97b0-d798-4e6c-2d42-08d4c7ad36e2","organization_name":"Mojave National Preserve","update_date":"2023-04-28T06:37:22.497934Z","update_frequency":150,"contact_name":"Mojave National Preserve","contact_email":"MOJA_Superintendent@nps.gov"},{"data_source_id":"78c13b90-565d-4167-2bd3-08d4c7ad36e2","organization_name":"Acadia National Park","update_date":"2023-09-29T09:50:50.496436Z","contact_name":"Acadia National Park","contact_email":"ACAD_Superintendent@nps.gov"},{"data_source_id":"82d0450b-aba8-46e4-2c62-08d4c7ad36e2","organization_name":"Dinosaur National Monument","update_date":"2023-10-03T11:13:41.534676Z","contact_name":"Dinosaur National Monument","contact_email":"DINO_Superintendent@nps.gov"},{"data_source_id":"c96651fe-ac5b-4213-

In [7]:
data['road_event_feed_info']

AttributeError: 'str' object has no attribute 'road_event_feed_info'

In [13]:
df = json.loads(data)


In [17]:
df["road_event_feed_info"]

{'publisher': 'National Park Service',
 'version': '4.1',
 'update_date': '2023-04-28T06:37:22.497618Z',
 'update_frequency': 60,
 'contact_name': 'National Park Service',
 'contact_email': 'asknps@nps.gov',
 'data_sources': [{'data_source_id': '40ce97b0-d798-4e6c-2d42-08d4c7ad36e2',
   'organization_name': 'Mojave National Preserve',
   'update_date': '2023-04-28T06:37:22.497934Z',
   'update_frequency': 150,
   'contact_name': 'Mojave National Preserve',
   'contact_email': 'MOJA_Superintendent@nps.gov'},
  {'data_source_id': '78c13b90-565d-4167-2bd3-08d4c7ad36e2',
   'organization_name': 'Acadia National Park',
   'update_date': '2023-09-29T09:50:50.496436Z',
   'contact_name': 'Acadia National Park',
   'contact_email': 'ACAD_Superintendent@nps.gov'},
  {'data_source_id': '82d0450b-aba8-46e4-2c62-08d4c7ad36e2',
   'organization_name': 'Dinosaur National Monument',
   'update_date': '2023-10-03T11:13:41.534676Z',
   'contact_name': 'Dinosaur National Monument',
   'contact_email': '

In [21]:
df["features"]['properties']

TypeError: list indices must be integers or slices, not str

In [25]:
for x in df["features"]:
    print(x['properties']['core_details'])

{'name': 'Cima Road Closed to ALL VEHICLES due to Construction', 'data_source_id': '40ce97b0-d798-4e6c-2d42-08d4c7ad36e2', 'event_type': 'work-zone', 'road_names': ['Cima Road'], 'direction': 'undefined', 'description': 'Cima Road is undergoing a major renovation for visitor and wildlife safety upgrades. Please do not drive around barricades or cross this active construction zone.'}
{'name': 'Expect carriage road construction between intersections 10 and 11', 'data_source_id': '78c13b90-565d-4167-2bd3-08d4c7ad36e2', 'event_type': 'work-zone', 'road_names': ['Carriage Road between intersections 10 and 11'], 'direction': 'unknown', 'description': 'Watch for heavy equipment and rough surfaces on carriage road between intersections 10 and 11 near Gilmore Meadow through October. Horses and carriages should avoid the area, which may be impassable at times. Bikers will need to dismount and walk.'}
{'name': 'Blasting and heavy construction at Park Headquarters', 'data_source_id': '78c13b90-565